# Predicting Newsletter Subscription Tendency: kNN Classification Model for MineCraft Player Engagement

Please install and run the libraries listed below in advance.

In [ ]:
### Run this cell before continuing. 
library(tidyverse)
library(tidymodels)
options(repr.plot.width=9, repr.plot.height=8)

## 1. Introduction

To build up success of online research platforms, particularly platforms collecting data on player activity in a video game environment, hinges on effective communication with and retain the user base. For the UBC Computer Science research group led by Frank Wood, which operates the MineCraft server, identifying the player characteristics that predict engagement and newsletter subscription is vital for optimizing recruitment and outreach strategies. 


The core question is: **What player characteristics are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?** More specifically: **How might a player's experience and total hours played predict their tendency to subscribe to a newsletter? And how does this differ between players of various ages?** 

This project uses the dataset `players.csv`, which contains unique player records with their descriptive features such as age and gender (see **Table 1** for a detailed description of each variable). We will begin by outlining the necessary data wrangling and cleaning procedures to address missing values (NA) and properly structure the dataset for modeling. Next, exploratory data analysis will be performed to visualize trends between player attributes (`Age`, `experience`, `played_hours`) and the binary subscription outcomes (`subscribe`). 
We will then implement and evaluate a k-Nearest Neighbors (kNN) classification model to predict players' subscription status, providing insights into which player profiles may be most responsive to targeted communication strategies. 

A potential limitation is the presence of missing data in the dataset. Additionally, `players.csv` is a relatively small dataset with only 196 observations, which may constrain the model's predictive accuracy. 

#### Table 1 - Variable Description of Dataset `players.csv`:
| variable | type | description |
|----------|------|-------------|
| experience | character | experience level (Beginner, Amateur, Regular, Pro, Veteran) |
| subscribe | logical | newsletter subscription (True/False) |
| hashedEmail | character | player identifier |
| played_hours | double | total hours played |
| name | character | player name |
| gender | character | player's gender |
| Age | double | age in years |


## 2. Methods and Results

The data analysis will be split up into five stages.
1. Loading the data and making it tidy
2. Creating new variables and replacing missing values
3. Conducting exploratory data analysis
4. Constructing a k-Nearest Neigbors classification model
5. Visualizing the results

### 2.1. Loading and Tidying the Dataset

In this step, we load in the dataset using the `read_csv` function and make some changes to the dataset.
First, we convert `subscribe` into a categorical factor which is necessary for classification. Then we create `age_group`, so that they could be compared during exploratory data analysis.

In [ ]:
players <- read_csv("https://raw.githubusercontent.com/eshaanmaniani/DSCI-Project-Planning/refs/heads/main/players.csv")
#Converts subscription to a factor and makes a new column which seperates age groups
players <- players |>
  mutate(
    subscribe = as.factor(subscribe),
    age_group = case_when(
      Age < 20 ~ "Under 20",
      Age >= 20 & Age <= 30 ~ "20 to 30",
      Age > 30 & Age <= 40 ~ "31 to 40",
      Age > 40 ~ "Over 40",
      TRUE ~ "Unknown"))

players

### 2.2. Summary Statistics

We compute the mean value for the quantitative variables (`played_hours` and `age`), and we present the values in **Table 2**. According to the table, most players are young and play for relatively few hours.

In [ ]:
#Finds the mean for each numerical column
numeric_mean <- players |>
  select(where(is.numeric)) |>
  summarise(across(everything(), ~ round(mean(.x, na.rm = TRUE), 2)))

numeric_mean

#### Table 2: Mean of Quantitative Variables
| quantitative variable | mean | 
|----------|------|
| played_hours | 5.85 | 
| Age | 21.14 | 


### 2.3. Exploratory Data Analysis

To visualize the relationship between player attributes (Age, Experience, Hours Played) and the binary subscription outcomes, we begin by exploring the subscription percentage across age groups in **Figure 1**. The bar chart shows that newsletter subscription is highly age-dependent. The highest subscription rates appear among the youngest players (under 30), with another noticeable increase among players over 40. In contrast, players between ages 31-40 show the lowest subscription percentages. This suggests that player characteristics, specifically age, meaningfully influence their likelihood of subscribing to newsletters.

In [ ]:
subscription_percentage <- ggplot(players, aes(x = age_group, fill = subscribe)) +
  geom_bar(position = "fill") +
  scale_y_continuous(labels = scales::percent) +
  labs(
    title = "Figure 1: Subscription Percentage by Age Group",
    x = "Age Group",
    y = "Percentage of Players",
    fill = "Subscribed"
  ) +
 theme(plot.title = element_text(size = 20))
subscription_percentage

We next explore the relationship between player age and total hours played using a scatterplot in **Figure 2**. The visualization illustrates that younger players (approximately 10-25 years old) display a wide range of total playtime, including several outlier values exceeding 150 hours played. In contrast, almost all players over 30 report fewer than 10 total hours played, suggesting lower engagement. Generally, the distribution is heavily skewed towards lower playtime, and the significant variability among young players shows that engagement is strongly age-dependent.

In [ ]:
players_vs_hours <- ggplot(players, aes(x = Age, y = played_hours)) +
  geom_point(alpha = 0.6) +
  labs(
    title = "Figure 2: Player Age vs Total Hours Played",
    x = "Age (Years)",
    y = "Total Hours Played"
  ) +
  theme(plot.title = element_text(size = 20))

players_vs_hours

To further examine the distribution of `played_hours`, we generated a histogram in **Figure 3**, which reveals a highly right-skewed distribution. The vast majority of players have logged fewer than 10 hours on the server, with only a small subset exceeding 50 hours, and a few extreme cases reaching over 150-200 hours. This indicates that the population mainly consists of casual players, while highly engaged players are comparatively rare. This skewness is important for modeling because it suggests that a small number of highly active players may disproportionately influence analyses involving total hours played.

In [ ]:
distribution_total_hours <- ggplot(players, aes(x = played_hours)) +
  geom_histogram(bins = 30) +
  labs(title = "Figure 3: Distribution of Total Hours Played",
    x = "Total Hours Played",
    y = "Count of Players") +
  theme(plot.title = element_text(size = 20))
distribution_total_hours

### 2.4. Building the k-Nearest Neighbors Model

We chose to use a K-Nearest Neighbours classification model to answer our question. We think that KNN is appropriate because the target variable is categorical (true/false). In this case, it will predict whether an individual subscribed to a newsletter based on the most common class of the nearest observations. The predictors are `experience` and `played_hours`, which must be scaled so that distances are comparable. Furthermore, we trained the model using a certain proportion of the dataset and evaluated the prediction accuracy on the remaining portion of the data, using a train/test split. 

This split ensures that the proportions subscribed are balanced across sets:

In [ ]:
set.seed(1)

player_split <- initial_split(players, prop = 0.75, strata = subscribe)
player_train <- training(player_split)
player_test  <- testing(player_split)

Since kNN relies on distance calculations, all predictors were scaled to ensure that variables on different units contributed equally to the model. Additionally, although `Age` showed significant patterns in the exploratory analysis, it was excluded from the final KNN model due to missing values and issues with scaling. Hence, the predictors used for classification were `experience` and `total_hours`.

In [ ]:
player_recipe <- recipe(subscribe ~ experience + played_hours, data = player_train) |>
  step_dummy(all_nominal_predictors()) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

In [ ]:
knn_spec <- nearest_neighbor(
  weight_func = "rectangular",
  neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("classification")

We use cross-validation to tune the optimal numbers of neighbors:

In [ ]:
player_vfold <- vfold_cv(player_train, v = 5, strata = subscribe)

k_vals <- tibble(neighbors = seq(1, 50, by = 2))

knn_results <- workflow() |>
  add_recipe(player_recipe) |>
  add_model(knn_spec) |>
  tune_grid(
    resamples = player_vfold,
    grid = k_vals
  ) |>
  collect_metrics()

We select the best value of k:

In [ ]:
best_k <- knn_results |>
  filter(.metric == "accuracy") |>
  arrange(desc(mean)) |>
  slice(1) |>
  pull(neighbors)

In [ ]:
knn_final_spec <- nearest_neighbor(
  weight_func = "rectangular",
  neighbors = best_k
) |>
  set_engine("kknn") |>
  set_mode("classification")

knn_final_fit <- workflow() |>
  add_recipe(player_recipe) |>
  add_model(knn_final_spec) |>
  fit(data = player_train)

This produces accuracy and confusion matrix values that indicate how well the KNN model predicts subscription status:

In [ ]:
player_test_predictions <- predict(knn_final_fit, player_test) |>
  bind_cols(player_test)

player_test_predictions |> accuracy(truth = subscribe, estimate = .pred_class)
player_test_predictions |> conf_mat(truth = subscribe, estimate = .pred_class)

The KNN model results indicate that the model accurately predicts subscriptions 67.3% of the time. Accuracy reflects the proportion of correct predictions on the test set. Since the dataset is highly imbalanced, accuracy alone may be misleading, which is why it is important to consider the confusion matrix. However, the confusion matrix indicates that out of all 49 predictions, 46 predictions were true while only 3 were predicted false, and all 3 were wrong. 

### 2.5. Visualizing the Results

Lastly, the result is visualized in **Figure 4**. It visually compares the actual subscription status of players with the predicted class labels from the knn model. While it correctly identifies many TRUE cases, it fails to correctly classify any FALSE cases, mislabeling nearly all non-subscribers as TRUE. This demonstrates that the model is heavily biased toward the majority class and struggles to distinguish between subscribers and non-subscribers.

In [ ]:
ggplot(player_test_predictions, aes(x = subscribe, fill = .pred_class)) +
  geom_bar(position = "dodge") +
  labs(
    title = "Figure 4: Comparison of True vs Predicted Subscription Status",
    x = "Actual Subscription Status",
    y = "Count of Players",
    fill = "Predicted Class"
  ) +
  scale_fill_manual(values = c("FALSE" = "tomato", "TRUE" = "steelblue")) +
  theme(plot.title = element_text(size = 20))


## 3. Discussion

**What we found:**

The KNN model achieved an overall accuracy of 67.3%, meaning it correctly predicted the subscription status for about two-thirds of the test set. However, the model is heavily biased toward the majority class (subscribers). In the training data (`player_train`), approximately 73% of cases are TRUE (subscribers) and 27% are FALSE (non-subscribers), as indicated by the results stored in `subscribe_proportions` below. This means a simple majority-class classifier would already achieve around 73% accuracy. The KNN classifier, with an accuracy of 67.3%, does not improve upon this baseline, indicating that it adds little predictive value in terms of overall accuracy.

Furthermore, if we consider FALSE (non-subscribers) as the class of interest (positive label), the classifier’s precision and recall are both 0%, despite the seemingly moderate accuracy. While the classifier correctly identifies almost all TRUE cases, it fails to classify any FALSE cases, mislabeling them as subscribers. The confusion matrix highlights this severe imbalance, and **Figure 4** visually demonstrates that the model cannot effectively distinguish non-subscribers from subscribers.

In [ ]:
subscribe_proportions <- player_train |>
                      group_by(subscribe) |>
                      summarize(n = n()) |>
                      mutate(percent = 100*n/nrow(player_train))
subscribe_proportions

**Whether this is what we expected to find:**

This result aligns with the expectation given the highly imbalanced dataset. To be more specific, KNN tends to favor the majority class when class distributions are uneven, especially when only two predictors (`experience` and `played_hours`) are used. While it was expected that the model would predict subscribers reasonably well, the complete failure to predict non-subscribers highlights the limitation of KNN in handling cases when classes are imbalanced.

**What impact could such findings have:**

The model’s bias toward the majority class limits its practical usefulness. In our case, the research team needs to target recruitment effectively and ensure they have enough resources to handle the player load. However, because our model predicts everyone as a subscriber, the team would likely overestimate demand, allocating more resources than necessary. This could be inefficient and costly, particularly for a university-run-project with limited funding and server capacity. 
Besides, since our model fails to identify non-subscribers, any recruitment strategies aimed at engaging them would be misguided, as the team would have no way knowing which players are less likely to participate. 

**What future questions could this lead to:**

- How can the model be improved to handle imbalanced data more effectively?
- Would incorporating additional features such as age and gender improve classification of non-subscribers?